In [21]:
import os
import sys

def find_project_root(target_folder="masteroppgave"):
    """Find the absolute path of a folder (e.g., 'masteroppgave') by searching upward."""
    currentdir = os.path.abspath("__file__")  # Get absolute script path
    while True:
        if os.path.basename(currentdir) == target_folder:
            return currentdir  # Found the target folder
        parentdir = os.path.dirname(currentdir)
        if parentdir == currentdir:  # Stop at filesystem root
            return None
        currentdir = parentdir  # Move one level up

# Find and append 'masteroppgave' to sys.path dynamically
project_root = find_project_root("masteroppgave")

if project_root:
    sys.path.append(project_root)
    print(f"Project root found: {project_root}")
else:
    raise RuntimeError("Could not find 'masteroppgave' directory")

Project root found: /Users/thomasnitsche/Developer/skole/master/masteroppgave


In [22]:
import folium
from folium.plugins import FeatureGroupSubGroup
import random
from schemes.lsh_disk import DiskLSH
from constants import  *
from utils.helpers import file_handler
import pprint

In [23]:

trajectories = file_handler.load_trajectory_files(["R_ABU.txt", "R_ARU.txt",  "R_ECN.txt"], ROME_OUTPUT_FOLDER_FULL) ## Reads a list of trajectory files, return a dictionary with the filename as key and coordinates as values

In [24]:
import folium
import random

# Rome coordinate bounds
R_MAX_LON = 12.53
R_MIN_LON = 12.44
R_MAX_LAT = 41.93
R_MIN_LAT = 41.88

def visualize_trajectories_with_bounds(trajectories):
    """
    Visualizes multiple trajectories on a Folium map with bounding box and point markers.

    Parameters:
    - trajectories (dict): Dictionary where keys are trajectory names, and values are lists of (lat, lon) coordinates.

    Returns:
    - A Folium map object.
    """

    # Get center of the map from first trajectory (if available)
    if not trajectories:
        raise ValueError("Trajectory dictionary is empty!")
    
    first_trajectory = list(trajectories.values())[0]
    center_lat, center_lon = first_trajectory[0]  # Center the map on first point

    # Initialize Folium map
    map_trajectories = folium.Map(location=[center_lat, center_lon], zoom_start=14, tiles="OpenStreetMap")

    # Add bounding box for Rome coordinates
    folium.Rectangle(
        bounds=[(R_MIN_LAT, R_MIN_LON), (R_MAX_LAT, R_MAX_LON)],
        color="black",
        weight=2,
        fill=True,
        fill_opacity=0.1,
        popup="Rome Bounding Box"
    ).add_to(map_trajectories)

    # Define a set of distinct colors for the trajectories
    traj_colors = [
        "blue", "green", "purple", "orange", "pink", "brown", "cyan", "magenta", "yellow", "lime"
    ]
    random.shuffle(traj_colors)  # Shuffle colors for randomness

    # Plot each trajectory with a unique color and mark points
    for idx, (traj_name, coords) in enumerate(trajectories.items()):
        traj_color = traj_colors[idx % len(traj_colors)]  # Assign a unique color

        # Add trajectory line
        folium.PolyLine(
            coords,
            color=traj_color,
            weight=4.5,
            opacity=1,
            popup=f"Trajectory: {traj_name}",
        ).add_to(map_trajectories)

        # Add markers for each point along the trajectory
        for lat, lon in coords:
            folium.CircleMarker(
                location=(lat, lon),
                radius=1,  # Small marker
                color="black",
                fill=True,
                fill_opacity=1,
                popup=f"Point: ({lat:.5f}, {lon:.5f})"
            ).add_to(map_trajectories)

    return map_trajectories
# Generate the map
map_trajectories = visualize_trajectories_with_bounds(trajectories)

# Save and open in browser
map_trajectories.save("trajectories.html")
